In [1]:
### imports
import warnings
warnings.simplefilter('ignore')
import itertools
import numpy as np
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from mliv.dgps import get_data, get_tau_fn, fn_dict
from mliv.neuralnet.utilities import mean_ci
from mliv.neuralnet import AGMMEarlyStop as AGMM
from mliv.neuralnet.moments import avg_small_diff
from sklearn.ensemble import RandomForestRegressor
import joblib
import pandas as pd
from collections import OrderedDict
import scipy


def plot_results(fname, n, iv_strength, dr, tmle, ipw, direct, true):
    plt.title(f'fname={fname}, n={n}, strength={iv_strength}, true={true:.3f}\n'
              f'dr: Cov={np.mean((dr[:, 1] <= true) & (true <= dr[:, 2])):.3f}, '
              f'rmse={np.sqrt(np.mean((dr[:, 0]-true)**2)):.3f}, '
              f'bias={np.mean((dr[:, 0]-true)):.3f}\n'
              f'tmle: Cov={np.mean((tmle[:, 1] <= true) & (true <= tmle[:, 2])):.3f}, '
              f'rmse={np.sqrt(np.mean((tmle[:, 0]-true)**2)):.3f}, '
              f'bias={np.mean((tmle[:, 0]-true)):.3f}\n'
              f'ipw: Cov={np.mean((ipw[:, 1] <= true) & (true <= ipw[:, 2])):.3f}, '
              f'rmse={np.sqrt(np.mean((ipw[:, 0]-true)**2)):.3f}, '
              f'bias={np.mean((ipw[:, 0]-true)):.3f}\n'
              f'direct: Cov={np.mean((direct[:, 1] <= true) & (true <= direct[:, 2])):.3f}, '
              f'rmse={np.sqrt(np.mean((direct[:, 0]-true)**2)):.3f}, '
              f'bias={np.mean((direct[:, 0]-true)):.3f}\n')
    plt.hist(dr[:, 0], label='dr')
    plt.hist(tmle[:, 0], label='tmle', alpha=.4)
    plt.hist(ipw[:, 0], label='ipw', alpha=.4)
    plt.hist(direct[:, 0], label='direct', alpha=.4)
    plt.legend()

n_z = 1
n_t = 1
dgp_num = 5
epsilon = 0.1 # average finite difference epsilon
moment_fn = lambda x, fn, device: avg_small_diff(x, fn, device, epsilon)

In [15]:
for clever in [False, True]:
    resd = {}
    for n_t in [1]:
        resd[n_t] = {}
        for fname in ['abs', '2dpoly', 'sigmoid', 'sin']:
            resd[n_t][fname] = OrderedDict()
            for n in [500, 1000, 2000]:
                lambda_l2_h = .1/n**(.9)
                print(lambda_l2_h)
                nkey = f'${n}$'
                resd[n_t][fname][nkey] = {}
                for iv_strength in [0.2, 0.5]:
                    ivkey = f'${iv_strength}$'
                    resd[n_t][fname][nkey][ivkey] = {}
                    for endogeneity_strength in [0.1, 0.3]:
                        true, results = joblib.load(f'res_fn_{fname}_n_{n}_n_t_{n_t}_stregth_{iv_strength}_{endogeneity_strength}_eps_{0.1}_clever_{clever}_l2h_{lambda_l2_h:.4f}.jbl')
                        ekey = f'${endogeneity_strength}$'
                        resd[n_t][fname][nkey][ivkey][ekey] = {}
                        for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
                            data = np.array([r[it] for r in results])
                            if method in ['dr', 'tmle']:
                                cov = f'{100*np.mean((data[:, 1] <= true) & (true <= data[:, 2])):.0f}'
                            else:
                                cov = 'NA'
                            resd[n_t][fname][nkey][ivkey][ekey][method] = {
                                            'cov': cov,
                                            'rmse': f'{np.sqrt(np.mean((data[:, 0]-true)**2)):.3f}',
                                            'bias': f'{np.abs(np.mean((data[:, 0]-true))):.3f}'}
                        resd[n_t][fname][nkey][ivkey][ekey] = pd.DataFrame(resd[n_t][fname][nkey][ivkey][ekey])
                    resd[n_t][fname][nkey][ivkey] = pd.concat(resd[n_t][fname][nkey][ivkey], sort=False)
                resd[n_t][fname][nkey] = pd.concat(resd[n_t][fname][nkey], sort=False)
            resd[n_t][fname] = pd.concat(resd[n_t][fname], sort=False)
        resd[n_t] = pd.concat(resd[n_t], sort=False)
    display(pd.concat(resd).droplevel(0).unstack(level=4))
    print(pd.concat(resd).droplevel(0).unstack(level=4).to_latex(bold_rows=True, multirow=True,
                                                    multicolumn=True, escape=False,
                                                    column_format='llll||lll|lll||lll|lll|',
                                                    multicolumn_format='c|'))

0.00037232911332721384
0.00019952623149688796
0.00010692345999911879
0.00037232911332721384
0.00019952623149688796
0.00010692345999911879
0.00037232911332721384
0.00019952623149688796
0.00010692345999911879
0.00037232911332721384
0.00019952623149688796
0.00010692345999911879


dr               tmle               ipw         \
                           cov   rmse   bias  cov   rmse   bias cov   rmse   
abs     $500$  $0.2$ $0.1$  96  0.151  0.010   93  0.160  0.009  NA  0.147   
                     $0.3$  94  0.152  0.016   91  0.173  0.019  NA  0.147   
               $0.5$ $0.1$  96  0.063  0.002   94  0.066  0.002  NA  0.060   
                     $0.3$  94  0.067  0.009   93  0.069  0.007  NA  0.061   
        $1000$ $0.2$ $0.1$  94  0.117  0.008   94  0.120  0.006  NA  0.117   
                     $0.3$  94  0.118  0.012   92  0.126  0.011  NA  0.117   
               $0.5$ $0.1$  94  0.049  0.002   94  0.049  0.002  NA  0.049   
                     $0.3$  94  0.048  0.004   93  0.049  0.003  NA  0.049   
        $2000$ $0.2$ $0.1$  95  0.080  0.010   95  0.080  0.009  NA  0.081   
                     $0.3$  95  0.082  0.015   94  0.085  0.012  NA  0.084   
               $0.5$ $0.1$  94  0.033  0.003   94  0.033  0.002  NA  0.035   
                     $0.3$  95  0.033  0.005   95  0.033  0.004  NA  0.035   
2dpoly  $500$  $0.2$ $0.1$  94  0.168  0.003   94  0.172  0.009  NA  0.186   
                     $0.3$  89  0.246  0.008   90  0.267  0.030  NA  0.275   
               $0.5$ $0.1$  95  0.065  0.002   95  0.065  0.004  NA  0.090   
                     $0.3$  92  0.082  0.005   92  0.082  0.006  NA  0.105   
        $1000$ $0.2$ $0.1$  95  0.127  0.004   94  0.128  0.004  NA  0.139   
                     $0.3$  92  0.181  0.012   92  0.190  0.016  NA  0.209   
               $0.5$ $0.1$  95  0.047  0.004   94  0.047  0.004  NA  0.055   
                     $0.3$  92  0.059  0.008   92  0.059  0.007  NA  0.071   
        $2000$ $0.2$ $0.1$  95  0.087  0.009   95  0.086  0.008  NA  0.101   
                     $0.3$  93  0.125  0.020   93  0.127  0.019  NA  0.157   
               $0.5$ $0.1$  96  0.032  0.003   96  0.032  0.002  NA  0.042   
                     $0.3$  94  0.040  0.004   94  0.040  0.004  NA  0.056   
sigmoid $500$  $0.2$ $0.1$  95  0.140  0.010   93  0.146  0.003  NA  0.134   
                     $0.3$  95  0.127  0.020   90  0.138  0.012  NA  0.130   
               $0.5$ $0.1$  94  0.054  0.009   93  0.056  0.011  NA  0.053   
                     $0.3$  94  0.049  0.004   92  0.051  0.006  NA  0.050   
        $1000$ $0.2$ $0.1$  94  0.104  0.002   93  0.106  0.001  NA  0.100   
                     $0.3$  94  0.096  0.010   92  0.101  0.006  NA  0.095   
               $0.5$ $0.1$  95  0.041  0.004   94  0.041  0.004  NA  0.039   
                     $0.3$  94  0.036  0.002   92  0.037  0.003  NA  0.034   
        $2000$ $0.2$ $0.1$  95  0.071  0.005   94  0.072  0.004  NA  0.067   
                     $0.3$  95  0.065  0.009   94  0.068  0.007  NA  0.063   
               $0.5$ $0.1$  95  0.028  0.001   95  0.028  0.001  NA  0.026   
                     $0.3$  95  0.025  0.002   95  0.025  0.001  NA  0.022   
sin     $500$  $0.2$ $0.1$  95  0.138  0.009   93  0.145  0.002  NA  0.134   
                     $0.3$  95  0.128  0.020   90  0.137  0.012  NA  0.128   
               $0.5$ $0.1$  94  0.057  0.017   91  0.059  0.021  NA  0.052   
                     $0.3$  93  0.050  0.012   91  0.052  0.014  NA  0.046   
        $1000$ $0.2$ $0.1$  94  0.103  0.002   94  0.105  0.000  NA  0.099   
                     $0.3$  94  0.094  0.009   91  0.100  0.005  NA  0.094   
               $0.5$ $0.1$  94  0.041  0.010   93  0.042  0.010  NA  0.039   
                     $0.3$  93  0.036  0.007   92  0.037  0.008  NA  0.033   
        $2000$ $0.2$ $0.1$  95  0.071  0.005   95  0.071  0.004  NA  0.066   
                     $0.3$  95  0.065  0.009   94  0.067  0.007  NA  0.063   
               $0.5$ $0.1$  95  0.027  0.001   95  0.027  0.002  NA  0.026   
                     $0.3$  95  0.024  0.000   94  0.024  0.001  NA  0.021   

                                  direct                
                             bias    cov   rmse   bias  
abs     $500$

\begin{tabular}{llll||lll|lll||lll|lll|}
\toprule
 &  &  &  & \multicolumn{3}{c|}{dr} & \multicolumn{3}{c|}{tmle} & \multicolumn{3}{c|}{ipw} & \multicolumn{3}{c|}{direct} \\
 &  &  &  & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias \\
\midrule
\multirow[t]{12}{*}{\textbf{abs}} & \multirow[t]{4}{*}{\textbf{$500$}} & \multirow[t]{2}{*}{\textbf{$0.2$}} & \textbf{$0.1$} & 96 & 0.151 & 0.010 & 93 & 0.160 & 0.009 & NA & 0.147 & 0.009 & NA & 0.129 & 0.002 \\
\textbf{} & \textbf{} & \textbf{} & \textbf{$0.3$} & 94 & 0.152 & 0.016 & 91 & 0.173 & 0.019 & NA & 0.147 & 0.017 & NA & 0.128 & 0.009 \\
\cline{3-16}
\textbf{} & \textbf{} & \multirow[t]{2}{*}{\textbf{$0.5$}} & \textbf{$0.1$} & 96 & 0.063 & 0.002 & 94 & 0.066 & 0.002 & NA & 0.060 & 0.002 & NA & 0.080 & 0.001 \\
\textbf{} & \textbf{} & \textbf{} & \textbf{$0.3$} & 94 & 0.067 & 0.009 & 93 & 0.069 & 0.007 & NA & 0.061 & 0.005 & NA & 0.075 & 0.003 \\
\cline{2-16} \cline{3-16}
\textbf{} & \multirow[t]{4}{*}{\te

dr               tmle               ipw         \
                           cov   rmse   bias  cov   rmse   bias cov   rmse   
abs     $500$  $0.2$ $0.1$  95  0.153  0.008   93  0.162  0.006  NA  0.147   
                     $0.3$  94  0.152  0.017   92  0.172  0.019  NA  0.147   
               $0.5$ $0.1$  94  0.066  0.003   93  0.068  0.003  NA  0.060   
                     $0.3$  95  0.064  0.007   94  0.067  0.006  NA  0.061   
        $1000$ $0.2$ $0.1$  94  0.117  0.008   94  0.119  0.006  NA  0.117   
                     $0.3$  94  0.118  0.012   92  0.126  0.011  NA  0.117   
               $0.5$ $0.1$  94  0.049  0.003   94  0.050  0.002  NA  0.049   
                     $0.3$  94  0.048  0.005   94  0.049  0.003  NA  0.049   
        $2000$ $0.2$ $0.1$  95  0.081  0.010   94  0.081  0.009  NA  0.081   
                     $0.3$  95  0.083  0.013   94  0.086  0.012  NA  0.084   
               $0.5$ $0.1$  95  0.033  0.003   94  0.033  0.002  NA  0.035   
                     $0.3$  95  0.033  0.005   95  0.033  0.004  NA  0.035   
2dpoly  $500$  $0.2$ $0.1$  94  0.169  0.003   94  0.174  0.008  NA  0.184   
                     $0.3$  90  0.244  0.008   91  0.264  0.028  NA  0.276   
               $0.5$ $0.1$  95  0.065  0.001   95  0.066  0.003  NA  0.089   
                     $0.3$  91  0.082  0.002   91  0.082  0.003  NA  0.104   
        $1000$ $0.2$ $0.1$  95  0.126  0.004   94  0.127  0.003  NA  0.139   
                     $0.3$  93  0.180  0.012   92  0.189  0.016  NA  0.209   
               $0.5$ $0.1$  94  0.047  0.004   94  0.047  0.003  NA  0.057   
                     $0.3$  93  0.058  0.006   92  0.059  0.005  NA  0.072   
        $2000$ $0.2$ $0.1$  96  0.087  0.009   96  0.086  0.008  NA  0.101   
                     $0.3$  93  0.126  0.024   94  0.128  0.023  NA  0.158   
               $0.5$ $0.1$  96  0.032  0.003   96  0.032  0.003  NA  0.042   
                     $0.3$  94  0.039  0.005   94  0.040  0.005  NA  0.055   
sigmoid $500$  $0.2$ $0.1$  95  0.138  0.009   94  0.144  0.002  NA  0.134   
                     $0.3$  95  0.126  0.018   91  0.138  0.011  NA  0.129   
               $0.5$ $0.1$  94  0.054  0.009   93  0.056  0.011  NA  0.055   
                     $0.3$  93  0.049  0.004   91  0.051  0.006  NA  0.048   
        $1000$ $0.2$ $0.1$  94  0.103  0.002   93  0.106  0.001  NA  0.100   
                     $0.3$  94  0.095  0.009   92  0.101  0.006  NA  0.095   
               $0.5$ $0.1$  95  0.041  0.005   93  0.041  0.005  NA  0.039   
                     $0.3$  94  0.036  0.002   93  0.037  0.003  NA  0.033   
        $2000$ $0.2$ $0.1$  95  0.071  0.005   95  0.071  0.003  NA  0.066   
                     $0.3$  95  0.065  0.009   94  0.068  0.007  NA  0.063   
               $0.5$ $0.1$  95  0.028  0.001   95  0.028  0.001  NA  0.026   
                     $0.3$  95  0.025  0.002   95  0.025  0.001  NA  0.022   
sin     $500$  $0.2$ $0.1$  95  0.139  0.010   93  0.145  0.004  NA  0.136   
                     $0.3$  95  0.127  0.018   91  0.138  0.011  NA  0.128   
               $0.5$ $0.1$  93  0.057  0.019   91  0.059  0.021  NA  0.052   
                     $0.3$  92  0.050  0.014   90  0.053  0.016  NA  0.046   
        $1000$ $0.2$ $0.1$  94  0.103  0.003   93  0.105  0.000  NA  0.100   
                     $0.3$  95  0.094  0.008   92  0.099  0.005  NA  0.094   
               $0.5$ $0.1$  93  0.042  0.010   93  0.042  0.011  NA  0.039   
                     $0.3$  93  0.036  0.007   92  0.037  0.007  NA  0.033   
        $2000$ $0.2$ $0.1$  95  0.070  0.005   95  0.071  0.004  NA  0.066   
                     $0.3$  95  0.065  0.008   93  0.067  0.007  NA  0.063   
               $0.5$ $0.1$  95  0.027  0.001   95  0.028  0.002  NA  0.026   
                     $0.3$  95  0.024  0.000   95  0.024  0.000  NA  0.021   

                                  direct                
                             bias    cov   rmse   bias  
abs     $500$

\begin{tabular}{llll||lll|lll||lll|lll|}
\toprule
 &  &  &  & \multicolumn{3}{c|}{dr} & \multicolumn{3}{c|}{tmle} & \multicolumn{3}{c|}{ipw} & \multicolumn{3}{c|}{direct} \\
 &  &  &  & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias \\
\midrule
\multirow[t]{12}{*}{\textbf{abs}} & \multirow[t]{4}{*}{\textbf{$500$}} & \multirow[t]{2}{*}{\textbf{$0.2$}} & \textbf{$0.1$} & 95 & 0.153 & 0.008 & 93 & 0.162 & 0.006 & NA & 0.147 & 0.009 & NA & 0.129 & 0.003 \\
\textbf{} & \textbf{} & \textbf{} & \textbf{$0.3$} & 94 & 0.152 & 0.017 & 92 & 0.172 & 0.019 & NA & 0.147 & 0.017 & NA & 0.117 & 0.006 \\
\cline{3-16}
\textbf{} & \textbf{} & \multirow[t]{2}{*}{\textbf{$0.5$}} & \textbf{$0.1$} & 94 & 0.066 & 0.003 & 93 & 0.068 & 0.003 & NA & 0.060 & 0.002 & NA & 0.074 & 0.001 \\
\textbf{} & \textbf{} & \textbf{} & \textbf{$0.3$} & 95 & 0.064 & 0.007 & 94 & 0.067 & 0.006 & NA & 0.061 & 0.005 & NA & 0.069 & 0.000 \\
\cline{2-16} \cline{3-16}
\textbf{} & \multirow[t]{4}{*}{\te

In [18]:
for clever in [False]:
    resd = {}
    for n_t in [1]:
        resd[n_t] = {}
        for fname in ['2dpoly']:
            resd[n_t][fname] = OrderedDict()
            for n in [2000, 20000]:
                lambda_l2_h = .1/n**(.9)
                nkey = f'${n}$'
                resd[n_t][fname][nkey] = {}
                for iv_strength in [0.05, 0.1]:
                    ivkey = f'${iv_strength}$'
                    resd[n_t][fname][nkey][ivkey] = {}
                    for endogeneity_strength in [0.05, 0.1, 0.3]:
                        true, results = joblib.load(f'res_fn_{fname}_n_{n}_n_t_{n_t}_stregth_{iv_strength}_{endogeneity_strength}_eps_{0.1}_clever_{clever}_l2h_{lambda_l2_h:.4f}.jbl')
                    
                        ekey = f'${endogeneity_strength}$'
                        resd[n_t][fname][nkey][ivkey][ekey] = {}
                        for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
                            data = np.array([r[it] for r in results])
                            if method in ['dr', 'tmle']:
                                cov = f'{100*np.mean((data[:, 1] <= true) & (true <= data[:, 2])):.0f}'
                            else:
                                cov = 'NA'
                            resd[n_t][fname][nkey][ivkey][ekey][method] = {
                                            'cov': cov,
                                            'rmse': f'{np.sqrt(np.mean((data[:, 0]-true)**2)):.3f}',
                                            'bias': f'{np.abs(np.mean((data[:, 0]-true))):.3f}'}
                        resd[n_t][fname][nkey][ivkey][ekey] = pd.DataFrame(resd[n_t][fname][nkey][ivkey][ekey])
                    resd[n_t][fname][nkey][ivkey] = pd.concat(resd[n_t][fname][nkey][ivkey], sort=False)
                resd[n_t][fname][nkey] = pd.concat(resd[n_t][fname][nkey], sort=False)
            resd[n_t][fname] = pd.concat(resd[n_t][fname], sort=False)
        resd[n_t] = pd.concat(resd[n_t], sort=False)
    display(pd.concat(resd).droplevel(0).unstack(level=4))
    print(pd.concat(resd).droplevel(0).unstack(level=4).to_latex(bold_rows=True, multirow=True,
                                                    multicolumn=True, escape=False,
                                                    column_format='llll||lll|lll|lll|lll|',
                                                    multicolumn_format='c|'))

dr               tmle               ipw         \
                             cov   rmse   bias  cov   rmse   bias cov   rmse   
2dpoly $2000$  $0.05$ $0.05$  94  0.355  0.031   94  0.359  0.034  NA  0.358   
                      $0.1$   92  0.397  0.038   94  0.423  0.055  NA  0.394   
                      $0.3$   88  0.728  0.027   85  6.251  0.522  NA  0.599   
               $0.1$  $0.05$  95  0.171  0.012   95  0.170  0.011  NA  0.181   
                      $0.1$   93  0.189  0.019   94  0.189  0.018  NA  0.203   
                      $0.3$   92  0.303  0.030   92  0.339  0.056  NA  0.308   
       $20000$ $0.05$ $0.05$  94  0.116  0.004   95  0.113  0.001  NA  0.136   
                      $0.1$   94  0.131  0.005   95  0.130  0.001  NA  0.149   
                      $0.3$   95  0.259  0.005   94  0.281  0.018  NA  0.246   
               $0.1$  $0.05$  94  0.054  0.001   95  0.054  0.001  NA  0.063   
                      $0.1$   94  0.060  0.001   95  0.059  0.002  NA  0.073   
                      $0.3$   95  0.098  0.005   95  0.098  0.003  NA  0.127   

                                    direct                
                               bias    cov   rmse   bias  
2dpoly $2000$  $0.05$ $0.05$  0.015     NA  0.343  0.215  
                      $0.1$   0.003     NA  0.372  0.243  
                      $0.3$   0.176     NA  0.527  0.395  
               $0.1$  $0.05$  0.004     NA  0.161  0.078  
                      $0.1$   0.004     NA  0.175  0.086  
                      $0.3$   0.054     NA  0.265  0.147  
       $20000$ $0.05$ $0.05$  0.014     NA  0.118  0.078  
                      $0.1$   0.003     NA  0.128  0.085  
                      $0.3$   0.029     NA  0.186  0.129  
               $0.1$  $0.05$  0.001     NA  0.045  0.016  
                      $0.1$   0.000     NA  0.048  0.016  
                      $0.3$   0.004     NA  0.069  0.017

\begin{tabular}{llll||lll|lll|lll|lll|}
\toprule
 &  &  &  & \multicolumn{3}{c|}{dr} & \multicolumn{3}{c|}{tmle} & \multicolumn{3}{c|}{ipw} & \multicolumn{3}{c|}{direct} \\
 &  &  &  & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias \\
\midrule
\multirow[t]{12}{*}{\textbf{2dpoly}} & \multirow[t]{6}{*}{\textbf{$2000$}} & \multirow[t]{3}{*}{\textbf{$0.05$}} & \textbf{$0.05$} & 94 & 0.355 & 0.031 & 94 & 0.359 & 0.034 & NA & 0.358 & 0.015 & NA & 0.343 & 0.215 \\
\textbf{} & \textbf{} & \textbf{} & \textbf{$0.1$} & 92 & 0.397 & 0.038 & 94 & 0.423 & 0.055 & NA & 0.394 & 0.003 & NA & 0.372 & 0.243 \\
\textbf{} & \textbf{} & \textbf{} & \textbf{$0.3$} & 88 & 0.728 & 0.027 & 85 & 6.251 & 0.522 & NA & 0.599 & 0.176 & NA & 0.527 & 0.395 \\
\cline{3-16}
\textbf{} & \textbf{} & \multirow[t]{3}{*}{\textbf{$0.1$}} & \textbf{$0.05$} & 95 & 0.171 & 0.012 & 95 & 0.170 & 0.011 & NA & 0.181 & 0.004 & NA & 0.161 & 0.078 \\
\textbf{} & \textbf{} & \textbf{} & \textbf{$0.1$} & 9

In [29]:
endogeneity_strength = 0.3
fname = 'cct'
for clever in [False]:
    resd = {}
    for n_t in [0, 5, 10]:
        resd[f'${n_t}$'] = {}
        for n in [1000, 5000]:
            lambda_l2_h = .1/n**(.9)
            nkey = f'${n}$'
            resd[f'${n_t}$'][nkey] = {}
            for iv_strength in [0.0, 0.5]:
                if n_t == 0 and iv_strength == 0.5:
                    continue

                true, results = joblib.load(f'res_fn_{fname}_n_{n}_n_t_{n_t}_stregth_{iv_strength}_{endogeneity_strength}_eps_{0.1}_clever_{clever}_l2h_{lambda_l2_h:.4f}.jbl')
                ivkey = f"${iv_strength}$"
                resd[f'${n_t}$'][nkey][ivkey] = {}
                for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
                    if method == 'ipw':
                        continue
                    data = np.array([r[it] for r in results])
                    if method in ['dr', 'tmle']:
                        cov = f'{100*np.mean((data[:, 1] <= true) & (true <= data[:, 2])):.0f}'
                    else:
                        cov = 'NA'
                    resd[f'${n_t}$'][nkey][ivkey][method] = {
                                    'cov': cov,
                                    'rmse': f'{np.sqrt(np.mean((data[:, 0]-true)**2)):.3f}',
                                    'bias': f'{np.abs(np.mean((data[:, 0]-true))):.3f}'}
                resd[f'${n_t}$'][nkey][ivkey] = pd.DataFrame(resd[f'${n_t}$'][nkey][ivkey])
            resd[f'${n_t}$'][nkey] = pd.concat(resd[f'${n_t}$'][nkey], sort=False)
        resd[f'${n_t}$'] = pd.concat(resd[f'${n_t}$'], sort=False)
    display(pd.concat(resd).unstack(level=3))
    print(pd.concat(resd).unstack(level=3).to_latex(bold_rows=True, multirow=True,
                                                    multicolumn=True, escape=False,
                                                    column_format='lll||lll|lll|lll|',
                                                    multicolumn_format='c|'))

dr               tmle               direct              
                  cov   rmse   bias  cov   rmse   bias    cov   rmse   bias
$0$  $1000$ $0.0$  92  0.084  0.027   92  0.077  0.029     NA  0.235  0.081
     $5000$ $0.0$  94  0.032  0.008   94  0.034  0.010     NA  0.027  0.013
$5$  $1000$ $0.0$  86  0.181  0.050   82  0.202  0.039     NA  0.313  0.131
            $0.5$  57  0.324  0.213   50  0.405  0.262     NA  0.331  0.020
     $5000$ $0.0$  55  0.143  0.092   53  0.161  0.105     NA  0.065  0.006
            $0.5$  42  0.142  0.097   44  0.146  0.084     NA  0.173  0.160
$10$ $1000$ $0.0$  86  0.139  0.011   80  0.164  0.025     NA  0.274  0.010
            $0.5$  62  0.241  0.116   56  0.317  0.155     NA  0.352  0.027
     $5000$ $0.0$  90  0.027  0.008   80  0.044  0.026     NA  0.074  0.071
            $0.5$  48  0.120  0.085   57  0.117  0.058     NA  0.180  0.157

\begin{tabular}{lll||lll|lll|lll|}
\toprule
 &  &  & \multicolumn{3}{c|}{dr} & \multicolumn{3}{c|}{tmle} & \multicolumn{3}{c|}{direct} \\
 &  &  & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias \\
\midrule
\multirow[t]{2}{*}{\textbf{$0$}} & \textbf{$1000$} & \textbf{$0.0$} & 92 & 0.084 & 0.027 & 92 & 0.077 & 0.029 & NA & 0.235 & 0.081 \\
\cline{2-12}
\textbf{} & \textbf{$5000$} & \textbf{$0.0$} & 94 & 0.032 & 0.008 & 94 & 0.034 & 0.010 & NA & 0.027 & 0.013 \\
\cline{1-12} \cline{2-12}
\multirow[t]{4}{*}{\textbf{$5$}} & \multirow[t]{2}{*}{\textbf{$1000$}} & \textbf{$0.0$} & 86 & 0.181 & 0.050 & 82 & 0.202 & 0.039 & NA & 0.313 & 0.131 \\
\textbf{} & \textbf{} & \textbf{$0.5$} & 57 & 0.324 & 0.213 & 50 & 0.405 & 0.262 & NA & 0.331 & 0.020 \\
\cline{2-12}
\textbf{} & \multirow[t]{2}{*}{\textbf{$5000$}} & \textbf{$0.0$} & 55 & 0.143 & 0.092 & 53 & 0.161 & 0.105 & NA & 0.065 & 0.006 \\
\textbf{} & \textbf{} & \textbf{$0.5$} & 42 & 0.142 & 0.097 & 44 & 0.146 & 0.084 & NA & 0.173 & 

In [28]:
endogeneity_strength = 0.3
fname = 'cct'
for clever in [False]:
    resd = {}
    for n_t in [0, 5, 10]:
        resd[f'${n_t}$'] = {}
        for n in [1000, 5000]:
            lambda_l2_h = .1/n**(.9)
            nkey = f'${n}$'
            resd[f'${n_t}$'][nkey] = {}
            for iv_strength in [0.0, 0.5]:
                if n_t == 0 and iv_strength == 0.5:
                    continue

                true, results = joblib.load(f'res_fn_{fname}_n_{n}_n_t_{n_t}_stregth_{iv_strength}_{endogeneity_strength}_eps_{0.1}_clever_{clever}_l2h_{lambda_l2_h:.4f}.jbl')
                ivkey = f"${iv_strength}$"
                resd[f'${n_t}$'][nkey][ivkey] = {}
                for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
                    if method == 'ipw':
                        continue
                    data = np.array([r[it] for r in results])
                    confidence = .95
                    se = (data[:, 2] - data[:, 0]) / scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
                    confidence = .99
                    data[:, 1] = data[:, 0] - se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
                    data[:, 2] = data[:, 0] + se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
                    if method in ['dr', 'tmle']:
                        cov = f'{100*np.mean((data[:, 1] <= true) & (true <= data[:, 2])):.0f}'
                    else:
                        cov = 'NA'
                    resd[f'${n_t}$'][nkey][ivkey][method] = {
                                    'cov': cov,
                                    'rmse': f'{np.sqrt(np.mean((data[:, 0]-true)**2)):.3f}',
                                    'bias': f'{np.abs(np.mean((data[:, 0]-true))):.3f}'}
                resd[f'${n_t}$'][nkey][ivkey] = pd.DataFrame(resd[f'${n_t}$'][nkey][ivkey])
            resd[f'${n_t}$'][nkey] = pd.concat(resd[f'${n_t}$'][nkey], sort=False)
        resd[f'${n_t}$'] = pd.concat(resd[f'${n_t}$'], sort=False)
    display(pd.concat(resd).unstack(level=3))
    print(pd.concat(resd).unstack(level=3).to_latex(bold_rows=True, multirow=True,
                                                    multicolumn=True, escape=False,
                                                    column_format='lll||lll|lll|lll|',
                                                    multicolumn_format='c|'))

dr               tmle               direct              
                  cov   rmse   bias  cov   rmse   bias    cov   rmse   bias
$0$  $1000$ $0.0$  98  0.084  0.027   98  0.077  0.029     NA  0.235  0.081
     $5000$ $0.0$  99  0.032  0.008   99  0.034  0.010     NA  0.027  0.013
$5$  $1000$ $0.0$  95  0.181  0.050   92  0.202  0.039     NA  0.313  0.131
            $0.5$  73  0.324  0.213   64  0.405  0.262     NA  0.331  0.020
     $5000$ $0.0$  68  0.143  0.092   66  0.161  0.105     NA  0.065  0.006
            $0.5$  54  0.142  0.097   56  0.146  0.084     NA  0.173  0.160
$10$ $1000$ $0.0$  94  0.139  0.011   90  0.164  0.025     NA  0.274  0.010
            $0.5$  74  0.241  0.116   68  0.317  0.155     NA  0.352  0.027
     $5000$ $0.0$  97  0.027  0.008   90  0.044  0.026     NA  0.074  0.071
            $0.5$  61  0.120  0.085   68  0.117  0.058     NA  0.180  0.157

\begin{tabular}{lll||lll|lll|lll|}
\toprule
 &  &  & \multicolumn{3}{c|}{dr} & \multicolumn{3}{c|}{tmle} & \multicolumn{3}{c|}{direct} \\
 &  &  & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias \\
\midrule
\multirow[t]{2}{*}{\textbf{$0$}} & \textbf{$1000$} & \textbf{$0.0$} & 98 & 0.084 & 0.027 & 98 & 0.077 & 0.029 & NA & 0.235 & 0.081 \\
\cline{2-12}
\textbf{} & \textbf{$5000$} & \textbf{$0.0$} & 99 & 0.032 & 0.008 & 99 & 0.034 & 0.010 & NA & 0.027 & 0.013 \\
\cline{1-12} \cline{2-12}
\multirow[t]{4}{*}{\textbf{$5$}} & \multirow[t]{2}{*}{\textbf{$1000$}} & \textbf{$0.0$} & 95 & 0.181 & 0.050 & 92 & 0.202 & 0.039 & NA & 0.313 & 0.131 \\
\textbf{} & \textbf{} & \textbf{$0.5$} & 73 & 0.324 & 0.213 & 64 & 0.405 & 0.262 & NA & 0.331 & 0.020 \\
\cline{2-12}
\textbf{} & \multirow[t]{2}{*}{\textbf{$5000$}} & \textbf{$0.0$} & 68 & 0.143 & 0.092 & 66 & 0.161 & 0.105 & NA & 0.065 & 0.006 \\
\textbf{} & \textbf{} & \textbf{$0.5$} & 54 & 0.142 & 0.097 & 56 & 0.146 & 0.084 & NA & 0.173 & 